In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
##########################################
#    Aggregates SNR (.dat), latency (.dat), and ROI photos (png)
#            scraped from PhotoZ
#    Given ROI files in format slice#_rois.dat 

In [2]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
csv_prefix = 'scnn1a_all_slices.csv'
csv_columns = ['Date',
             'Slice/Loc/Rec',
             'Stim Location',
             'L4 barrel 1 Latency',
             'L4 barrel 2 Latency',
             'L4 barrel 3 Latency',
             'L4 barrel 1 SNR',
             'L4 barrel 2 SNR',
             'L4 barrel 3 SNR',
             'ROI jpg']

enable_photoZ_interaction = True
initialize_photoZ = False

# Load reference data for all slices
reference_file = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/scnn1a_all.csv"
reference_df = pd.read_csv(reference_file)
ref_cols = ['Age', 
          'Sex', 
          "Infusion", 
          'Stim (uA)']

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_prefix)
df = pd.DataFrame(df)

In [3]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [ ]:
for subdir, dirs, files in os.walk(data_dir):
    zda_files = []
    roi_files = {}  # map slice number to ROIs file
    
    if 'selected_zda' not in subdir:
        continue
    
    # locate important file names
    for f in files:
        if f.endswith(".zda"):
            zda_files.append(f)
        if 'slice' in f[-17:] and f.endswith('_rois.dat'):
            slice_no = int(f.split("_rois.dat")[0][-1])
            roi_files[slice_no] = f
    
    if len(roi_files) < 1 or len(zda_files) < 1:
        continue
    
    for zda_file in zda_files:
        rec_id = zda_file.split('.')[0]
            
        # determine date and slice, loc, rec
        
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            aPhz.open_zda_file(subdir + "/" + zda_file)
        
        date = subdir.split("selected_zda")[0][-9:-1]
        date = [int(x) for x in date.split("-")]
        date[2] += 2000  # full year format
        date = "/".join([str(d) for d in date])
        print(date)
        
        slice_no = int(rec_id.split("_")[0])
        roi_file = roi_files[slice_no]
        print("\n\nscraping data from ", zda_file, "using", roi_file)
        print("Date:", date)

        filename_end = "_" + rec_id + '.dat'

        # open roi file in photoZ
        if enable_photoZ_interaction:
            aPhz.select_roi_tab()
            aPhz.open_roi_file(roi_file)

        # save latency values from PhotoZ
        latency_filename = subdir + "/" + "latency_" + filename_end
        print("Latency file dst:", latency_filename)
        if enable_photoZ_interaction:
            aPhz.select_latency_trace_value()
            aPhz.save_trace_values(latency_filename)

        # save SNR values from PhotoZ
        snr_filename = subdir + "/" + "SNR_" + filename_end
        print("SNR file dst:", snr_filename)
        if enable_photoZ_interaction:
            aPhz.select_SNR_trace_value()
            aPhz.save_trace_values(snr_filename)


        # take JPG photo of ROIs so we know which measurements go where.
        rois_frame_filename = subdir + "/" + "ROIs_" + rec_id + ".jpg"
        if enable_photoZ_interaction:
            aPhz.save_map_jpeg(rois_frame_filename)

        intermed_files = [latency_filename, snr_filename, rois_frame_filename]
        column_groups = [
            ['L4 barrel 1 Latency', 'L4 barrel 2 Latency', 'L4 barrel 3 Latency'],
            ['L4 barrel 1 SNR', 'L4 barrel 2 SNR', 'L4 barrel 3 SNR'],
            ['ROI jpg']
        ]
        
        # look up stim location, age, sex, infusion from scnn1a_all.csv
        ref_row = reference_df[reference_df["Date"] == date][ref_cols]        
        new_row = {"Date" : date, 
                   "Slice/Loc/Rec": rec_id}
        for col in ref_cols:
            new_row[col] = ref_row.iloc[0][col]
        
        # open these various files and store them to the pandas dataframe
        # The first three ROIs of each should correspond to the 3 desired barrel ROIs
        # each barrel/value pair is a column in the target CSV
        for j in range(len(intermed_files)):
            if intermed_files[j].endswith(".dat"):
                intermed_df = pd.read_csv(intermed_files[j],
                                          sep='\t',
                                          header=None,
                                          names=['Index',  'Values'])
                for i in range(len(column_groups[j])):
                    lat_val = intermed_df['Values'][i]
                    new_row[column_groups[j][i]] = lat_val
            else:  # .jpg, store link as csv value
                new_row[column_groups[j][0]] = intermed_files[j]
        print(new_row)
        df = df.append(new_row, ignore_index=True)

        

        continue

        
'''        for i in range(len(intermed_files)):
            intermed_file = intermed_files[i]
            col_name = col_names[i]
            intermed_df = pd.read_csv(intermed_file,
                                     sep='\t',
                                     header=None,
                                     names=['Index',  'Values'])

            row_indices = df.index[(df['Date'] == date) 
             & (df['Laminar'].isin(lam_type))
             & (df['Slice_Loc_Run'] == rec_id)].tolist()

            if intermed_df['Values'].shape[0] == len(row_indices):
                for j in range(len(row_indices)):
                    row_idx = row_indices[j]
                    df.loc[row_idx,[col_name]] = intermed_df['Values'][j]
            else:
                print("Could not write values for " + " ".join(lam_type) 
                      + " for date " + date + " rec " + rec_id
                      + "col name" + col_name)
                print(df[(df['Date'] == date) 
                 & (df['Laminar'].isin(lam_type))
                 & (df['Slice_Loc_Run'] == rec_id)])
                print(intermed_df['Values'])
'''
        # print("\n\nNum empty cells ", df[['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']].count())
        #print("AFTER WRITE")
        #print(df[(df['Date'] == date) 
        #       & (df['Laminar'].isin(lam_type))
        #       & (df['Slice_Loc_Run'] == rec_id)][col_names])


2/28/2023


scraping data from  02_01_01.zda using slice2_rois.dat
Date: 2/28/2023
Latency file dst: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-28-23\selected_zda/latency__02_01_01.dat
SNR file dst: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-28-23\selected_zda/SNR__02_01_01.dat
{'Date': '2/28/2023', 'Slice/Loc/Rec': '02_01_01', 'Age': 58, 'Sex': 'M', 'Infusion': nan, 'Stim (uA)': '100', 'L4 barrel 1 Latency': 50.3009, 'L4 barrel 2 Latency': 50.8406, 'L4 barrel 3 Latency': 49.3457, 'L4 barrel 1 SNR': 2.94039, 'L4 barrel 2 SNR': 10.6309, 'L4 barrel 3 SNR': 6.48439, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-28-23\\selected_zda/ROIs_02_01_01.jpg'}
2/28/2023


scraping data from  02_01_03.zda using slice2_rois.dat
Date: 2/28/2023
Latency file dst: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-28-23\selected_zda/latency__02_01_03.dat
SNR file dst: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-28-23\selected_zda

In [ ]:
print(df)

In [ ]:
# write the pandas dataframe back to csv
csv_filename = 'scnn1a_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)